In [1]:
import time
import hashlib
import pandas as pd

In [2]:
'''
Setting up the input data for the blocks 
Each row in this dataframe corresponds to a new block in the chain
The dataframe will also store the metadata about block once we are done mining each bloc
'''
block_data = pd.DataFrame(columns={'data','data_id','prev_block_id','curr_block_id', 'mining_time','nonce'})
block_data['data'] = ["Let's create the first block in the chain", "This is the second block", "This is the third block", "This is the fourth block"]
block_data

,data,prev_block_id,nonce,data_id,curr_block_id,mining_time
0,Let's create the first block in the chain,NaN,NaN,NaN,NaN,NaN
1,This is the second block,NaN,NaN,NaN,NaN,NaN
2,This is the third block,NaN,NaN,NaN,NaN,NaN
3,This is the fourth block,NaN,NaN,NaN,NaN,NaN


In [45]:
#How many leading zeros count (lzc) should be there in the acceptable hash for current block ID
'''User can change this, please note that the larger this number the longer it will take for the program to finish running'''
lzc = 3

In [46]:
'''
This function is called for each row (therefore each block)
::input::
data_to_hash: the data to be stored in the block
prev_block_id: the previous block's id, for the first block it will be empty
::output::
data_id: the Data ID for the datain the block
prev_block_id: the previous block ID
current_block_id: the current block ID
nonce: the nonce value, which also represent the number of attempts it took to mine the block
mining_time: the time in seconds that it took to mine the block
'''
def create_block(data_to_hash, prev_block_id=None):
    result = hashlib.md5(data_to_hash.encode())
    data_id = result.hexdigest()
    
    start_time = time.time()
    if prev_block_id:
        input_data = prev_block_id+'\n'+data_id
    else:
        input_data = data_id
    current_block_id = hashlib.md5(input_data.encode()).hexdigest()
    
    nonce = 0
    if current_block_id[:lzc]!='0'*lzc:
        while current_block_id[:lzc]!= '0'*lzc:
            nonced_data = input_data+'\n'+str(nonce)
            current_block_id = hashlib.md5(nonced_data.encode()).hexdigest()
            nonce = nonce+1
    mining_time = time.time() - start_time
    return data_id, prev_block_id, current_block_id, mining_time, nonce

In [47]:
'''Calling the block mining function for each row of the input'''
for index, row in block_data.iterrows():
    data_to_hash = row['data']
    if index!=0:
        prev_block_id = block_data.at[index-1,'curr_block_id']
        mining_output = create_block(data_to_hash,prev_block_id)
    else:
        mining_output = create_block(data_to_hash)
    row['data_id'] = mining_output[0]
    row['prev_block_id'] = mining_output[1]
    row['curr_block_id'] = mining_output[2]
    row['mining_time'] = mining_output[3]
    row['nonce'] = mining_output[4]

In [48]:
'''Display the output once all blocks are mined'''
block_data[['mining_time','nonce','data','data_id','prev_block_id','curr_block_id']]

,mining_time,nonce,data,data_id,prev_block_id,curr_block_id
0,0.00100017,540,Let's create the first block in the chain,bcfa87c919c4e7710975408c8971a5b1,None,0001a4fb396dfeab55a70aa0967b618e
1,0.0150037,8287,This is the second block,f983c34da708267a231b4b550d89090a,0001a4fb396dfeab55a70aa0967b618e,000e696bab2b4d7570fd0a8a509c86a3
2,0.000999928,527,This is the third block,5017c5fbf82a92b6ad94fb827c90390f,000e696bab2b4d7570fd0a8a509c86a3,0003e63fc9a23ce49af7a790be093ebd
3,0.00200105,1411,This is the fourth block,f2bfa3483b3d134247054ba9e7d2d7dc,0003e63fc9a23ce49af7a790be093ebd,000f46801330047b5998ba002f0327e2
